In [46]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np


In [47]:
data_df = pd.read_csv('columbus_data.csv')

In [48]:
display(data_df.describe())

,Unnamed: 0_x,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),...,educational_attainment_less_than_high_school_diploma,educational_attainment_high_school_graduate,educational_attainment_associate_degree,educational_attainment_bachelor_degree,educational_attainment_master_degree,educational_attainment_professional_school_degree,educational_attainment_doctorate_degree,school_enrollment_enrolled_in_public_school,school_enrollment_enrolled_in_private_school,school_enrollment_not_enrolled_in_school
count,3.907200e+04,39072.000000,39072.000000,39072.000000,13563.000000,13563.000000,39072.000000,39003.000000,28498.000000,38898.000000,...,34505.000000,34505.000000,34505.000000,34505.000000,34505.000000,34505.000000,34505.000000,34505.000000,34505.000000,34505.000000
mean,2.840297e+06,2.451397,39.881538,-83.389175,39.881442,-84.106651,0.466862,52.238415,44.335620,71.433184,...,1694.434024,6805.892972,910.722127,3371.662918,1336.499464,366.141806,263.542153,2941.552818,508.295001,475.774699
std,2.256009e+06,0.589459,1.329092,3.332932,1.945096,5.393807,1.206985,20.018690,24.255043,20.098558,...,1567.493676,5135.333019,721.879260,2660.608837,1127.922957,335.877138,248.038856,2630.848775,498.321099,427.173127
min,1.000000e+01,1.000000,29.663366,-109.424154,29.691592,-109.671261,0.000000,-20.000000,-41.000000,8.000000,...,48.000000,44.000000,6.000000,86.000000,7.000000,0.000000,0.000000,126.000000,9.000000,7.000000
25%,9.262720e+05,2.000000,39.951691,-83.023888,39.951887,-83.031940,0.000000,37.000000,26.900000,56.000000,...,607.000000,2875.000000,476.000000,1136.000000,353.000000,25.000000,48.000000,813.000000,164.000000,209.000000
50%,2.296901e+06,2.000000,39.974571,-82.991516,39.975452,-82.996823,0.000000,53.000000,41.200000,75.000000,...,912.000000,4713.000000,648.000000,2931.000000,1269.000000,387.000000,175.000000,1904.000000,276.000000,250.000000
75%,4.157509e+06,3.000000,40.014851,-82.956490,40.022829,-82.964410,0.401000,69.000000,65.000000,89.000000,...,2509.000000,9838.000000,1232.000000,4517.000000,1552.000000,562.000000,374.000000,4531.000000,864.000000,765.000000
max,7.727027e+06,4.000000,48.822446,-74.653181,48.836445,-74.653181,40.476000,99.000000,99.000000,100.000000,...,7200.000000,23623.000000,3253.000000,12321.000000,4475.000000,1642.000000,1164.000000,11343.000000,2184.000000,1799.000000


In [49]:
categorical_columns = data_df.select_dtypes(include=['object', 'category']).columns

In [50]:
display(categorical_columns)

Index(['ID', 'Source', 'Start_Time', 'End_Time', 'Description', 'Street',
       'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Wind_Direction',
       'Weather_Condition', 'Sunrise_Sunset', 'Civil_Twilight',
       'Nautical_Twilight', 'Astronomical_Twilight', 'Region', 'Neighbor_1',
       'Neighbor_2', 'Neighbor_3', 'Neighbor_4', 'Neighbor_5', 'Neighbor_6',
       'state', 'county', 'time_zone'],
      dtype='object')

In [51]:
columns_to_remove = ['Precipitation(in)', 'Wind_Chill(F)', 'End_Lat', 'End_Lng','Airport_Code']
data_df.drop(columns=list(columns_to_remove), inplace=True)

In [52]:
columns_to_fill = [
    'population',
    'population_density',
    'housing_units',
    'means_of_transportation_to_work_car_truck_van',
    'means_of_transportation_to_work_public_transportation',
    'means_of_transportation_to_work_taxicab',
    'means_of_transportation_to_work_motorcycle',
    'means_of_transportation_to_work_bicycle_walked_other_means',
    'means_of_transportation_to_work_worked_at_home',
    'travel_time_to_work_less_than_10_minutes',
    'travel_time_to_work_10_to_19_minutes',
    'travel_time_to_work_20_to_29_minutes',
    'travel_time_to_work_30_to_39_minutes',
    'travel_time_to_work_40_to_44_minutes',
    'travel_time_to_work_45_to_59_minutes',
    'travel_time_to_work_60_to_89_minutes',
    'travel_time_to_work_more_than_90_minutes',
    'Wind_Speed(mph)', 
    'Temperature(F)', 
    'Humidity(%)',
    'Pressure(in)',
    'Visibility(mi)'
]

# Replace missing values with the mean for each column in the list
for column in columns_to_fill:
    mean_value = data_df[column].mean()
    data_df[column] = data_df[column].fillna(mean_value)

In [53]:
categorical_columns_to_fill = [
    'Weather_Timestamp', 
    'Wind_Direction', 
    'Weather_Condition', 
    'Sunrise_Sunset',
    'Civil_Twilight',
    'Nautical_Twilight',
    'Astronomical_Twilight',
    'Street'
]
# Replace missing values with the mode for each categorical column in the list
for column in categorical_columns_to_fill:
    mode_value = data_df[column].mode()[0]  # mode() returns a Series, get the first value
    data_df[column] = data_df[column].fillna(mode_value)

In [54]:
columns_to_drop = [
    'zip_code', 'state', 'county', 'time_zone', 'latitude', 'longitude', 'median_home_value', 'land_area', 'water_area', 'occupied_housing_units', 'median_household_income', 
    'population_2005', 'population_2006', 'population_2007', 'population_2008', 'population_2009', 'population_2010', 'population_2011', 'population_2012', 'population_2013', 
    'population_2014', 'population_2015', 'population_2016', 'population_2017', 'population_2018', 'population_2019', 'population_2020', 'population_median_age', 
    'population_male_median_age', 'population_female_median_age', 'gender_male', 'gender_female', 'race_white', 'race_black_or_african_american', 
    'race_american_indian_or_alaskan_native', 'race_asian', 'race_native_hawaiian_and_other_pacific_islander', 'race_other_race', 'race_asian_two_or_more_races', 
    'family_type_husband_wife', 'gender_single_guardian', 'gender_singles', 'gender_singles_with_roommate', 'average_household_size', 'households_without_kids', 
    'households_with_kids', 'housing_type_in_occupied_housing_units', 'housing_type_correctional_facility_for_adults', 'housing_type_juvenile_facilities', 
    'housing_type_nursing_facilities', 'housing_type_other_institutional', 'housing_type_college_student_housing', 'housing_type_military_quarters', 
    'housing_type_other_noninstitutional', 'housing_built_year_1939_or_earlier', 'housing_built_year_1940s', 'housing_built_year_1950s', 'housing_built_year_1960s', 
    'housing_built_year_1970s', 'housing_built_year_1980s', 'housing_built_year_1990s', 'housing_built_year_2000s', 'housing_built_year_2010_or_after'
]

data_df.drop(columns=columns_to_drop, inplace=True)
additional_columns_to_drop = [
    'housing_occupancy_owned_households_with_mortgage',
    'housing_occupancy_owned_households_free_and_clear',
    'housing_occupancy_renter_occupied_households',
    'housing_occupancy_households_vacant',
    'vacancy_reasons_for_rent',
    'vacancy_reasons_rented_and_unoccupied',
    'vacancy_reasons_for_sale_only',
    'vacancy_reasons_sold_and_unoccupied',
    'vacancy_reasons_for_season_recreational_or_occasional_use',
    'vacancy_reasons_for_migrant_workers',
    'vacancy_reasons_vacant_for_other_reasons',
    'owner_occupied_home_values_from_$1_to_$24999_dollars',
    'owner_occupied_home_values_from_$25000_to_$49999_dollars',
    'owner_occupied_home_values_from_$50000_to_$99999_dollars',
    'owner_occupied_home_values_from_$100000_to_$149999_dollars',
    'owner_occupied_home_values_from_$150000_to_$199999_dollars',
    'owner_occupied_home_values_from_$200000_to_$399999_dollars',
    'owner_occupied_home_values_from_$400000_to_$749999_dollars',
    'owner_occupied_home_values_more_than_$750000_dollars',
    'rented_housing_number_of_rooms_studio',
    'rented_housing_number_of_rooms_1_bedroom',
    'rented_housing_number_of_rooms_2_bedroom',
    'rented_housing_number_of_rooms_3_or_more_bedroom',
    'employment_status_fulltime_with_earnings',
    'employment_status_parttime_with_earnings',
    'employment_status_no_earnings',
    'average_household_income_2005($)',
    'average_household_income_2006($)',
    'average_household_income_2007($)',
    'average_household_income_2008($)',
    'average_household_income_2009($)',
    'average_household_income_2010($)',
    'average_household_income_2011($)',
    'average_household_income_2012($)',
    'average_household_income_2013($)',
    'average_household_income_2014($)',
    'average_household_income_2015($)',
    'average_household_income_2016($)',
    'average_household_income_2017($)',
    'average_household_income_2018($)',
    'average_household_income_2019($)',
    'average_household_income_2020($)'
]

data_df.drop(columns=additional_columns_to_drop, inplace=True)
more_columns_to_drop = [
    'household_income_less_than_25000_dollars',
    'household_income_from_45000_to_59999_dollars',
    'household_income_from_60000_to_99999_dollars',
    'household_income_from_100000_to_149999_dollars',
    'household_income_from_150000_to_199999_dollars',
    'household_income_more_than_200000_dollars',
    'annual_individual_earnings_less_than_10000_dollars',
    'annual_individual_earnings_from_10000_to_19999_dollars',
    'annual_individual_earnings_from_20000_to_29999_dollars',
    'annual_individual_earnings_from_30000_to_39999_dollars',
    'annual_individual_earnings_from_40000_to_49999_dollars',
    'annual_individual_earnings_from_50000_to_64999_dollars',
    'annual_individual_earnings_from_65000_to_74999_dollars',
    'annual_individual_earnings_from_75000_to_99999_dollars',
    'annual_individual_earnings_more_than_100000_dollars'
]

data_df.drop(columns=more_columns_to_drop, inplace=True)
final_columns_to_drop = [
    'earnings_source_fulltime_with_earnings',
    'earnings_source_parttime_with_earnings',
    'earnings_source_no_earnings',
    'educational_attainment_less_than_high_school_diploma',
    'educational_attainment_high_school_graduate',
    'educational_attainment_associate_degree',
    'educational_attainment_bachelor_degree',
    'educational_attainment_master_degree',
    'educational_attainment_professional_school_degree',
    'educational_attainment_doctorate_degree',
    'school_enrollment_enrolled_in_public_school',
    'school_enrollment_enrolled_in_private_school',
    'school_enrollment_not_enrolled_in_school'
]

data_df.drop(columns=final_columns_to_drop, inplace=True)

In [55]:
# Set max rows displayed in output to unlimited
pd.set_option('display.max_rows', None)

#check for missing values
missing_values = data_df.isnull().sum()
display(missing_values)

Unnamed: 0_x                                                     0
ID                                                               0
Source                                                           0
Severity                                                         0
Start_Time                                                       0
End_Time                                                         0
Start_Lat                                                        0
Start_Lng                                                        0
Distance(mi)                                                     0
Description                                                      0
Street                                                           0
City                                                             0
County                                                           0
State                                                            0
Zipcode                                                       

In [56]:

data_df.to_csv('cleaned_data.csv', index=False)

In [57]:
def encode_data(data_df):
    # Normalize numerical data
    categorical_features = data_df.select_dtypes(exclude=[np.number]).columns.tolist()
    numeric_features = data_df.select_dtypes(include=[np.number]).columns.tolist()

    # Encode categorical features using oneHotEncoder, and standardize numerical features using StandardScaler
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(), categorical_features)
        ])

    # Encoded data is here
    data_df_transformed = preprocessor.fit_transform(data_df)
    
    return data_df_transformed

In [58]:
encoded_data=encode_data(data_df)
print(encoded_data)

  (0, 0)	-1.2590036362742831
  (0, 1)	0.9306995231445025
  (0, 2)	0.05362652113438939
  (0, 3)	0.08097546045083422
  (0, 4)	-0.37851978940179143
  (0, 5)	-0.83188959186054
  (0, 6)	1.0754640277338177
  (0, 7)	0.23009486916800972
  (0, 8)	-1.363607754039141
  (0, 9)	-0.6057722587969142
  (0, 10)	0.04550803026739024
  (0, 11)	0.07893129350400331
  (0, 12)	0.900329319308024
  (0, 14)	2.0401138979787268
  (0, 15)	-0.9514467079070255
  (0, 16)	2.106333816841539
  (0, 17)	1.9311333101692107
  (0, 18)	-0.6956462406397681
  (0, 19)	4.412931524680653
  (0, 20)	-0.07948133239716998
  (0, 21)	-0.02302238512861432
  (0, 22)	0.3122176709174046
  (0, 23)	1.5014833770720388
  (0, 24)	1.27430234875408
  (0, 25)	1.9932847928053947
  :	:
  (39071, 98148)	1.0
  (39071, 98202)	1.0
  (39071, 98204)	1.0
  (39071, 98205)	1.0
  (39071, 98207)	1.0
  (39071, 98209)	1.0
  (39071, 98211)	1.0
  (39071, 98213)	1.0
  (39071, 98215)	1.0
  (39071, 98216)	1.0
  (39071, 98218)	1.0
  (39071, 98220)	1.0
  (39071, 98222)	1

In [59]:
import torch
from torch_geometric.data import Data

#x = torch.tensor(data_df_transformed, dtype=torch.float)
#edge_index = torch.tensor(edge_index, dtype=torch.long)

# Create the Data object
#data = Data(x=x, edge_index=edge_index)